# NOAA Sea Lion Population Count Challenge
This notebook is a cnn solution built off of previous approaches using keras. The images are resized conformly to 512x512. 

In [1]:
import keras
import cv2
import sys
import os
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, GlobalAveragePooling2D, Input, concatenate
from keras.models import Model
from keras import backend as K
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator

%matplotlib notebook

Using TensorFlow backend.


## Image Preprocessing
There needs to be preprocessing to reduce the size of images to fit into memory. This notebook assumes the images are all resized to 512x512. This can be done using ImageMagik

```sh
mogrify -resize 512x512! -format png -path test_images_512x512 Test/*.jpg
```

<hr>

## Configuration

In [2]:
n_classes= 5                   # amount of lion classes
batch_size= 20                 #
epochs= 100                    # number of epochs (start @ 100) 150 better score
image_size= 512                # resized img 
model_name= 'cnn_regression'   # 

train_dir = '/media/bss/Ubuntu HDD/noaa-sealions/data_512/train'
validation_dir = '/media/bss/Ubuntu HDD/noaa-sealions/data_512/validation'
test_dir = '/media/bss/Ubuntu HDD/noaa-sealions/data_512/test'

In [3]:
# verify files
len(os.listdir(test_dir)), len(os.listdir(train_dir)), len(os.listdir(validation_dir))

(17745, 713, 179)

In [4]:
'''
Return a list of the miss classed images to remove
'''
def read_ignore_list():
    df_ignore = pd.read_csv('../data/miss_class.txt')
    return df_ignore['train_id'].tolist()

In [16]:
'''
Main Loading Function
'''
def load_data(dir_path):
    df = pd.read_csv('../data/train.csv')
    ignore_list = read_ignore_list()
    
    n_train_wValidation_images = 713
    n_train_images = 948
    
    image_list = []  # storage
    y_list = []      # storage
    
    for i in range(0, n_train_images):
        #if i not in ignore_list and i.endswith('.png'):
        img_path = os.path.join(dir_path, str(i) + '.png')
        img = cv2.imread(img_path)
        print('Image shape: ' + str(img.shape))

        image_list.append(img)

        row = df.ix[i]

        y_row = np.zeros((5))
        y_row[0] = row['adult_males']
        y_row[1] = row['subadult_males']
        y_row[2] = row['adult_females']
        y_row[3] = row['juveniles']
        y_row[4] = row['pups']
        y_list.append(y_row)
    x_train = np.asarray(image_list)
    y_train = np.asanyarray(y_list)
    
    print('X Train: ' + str(x_train.shape))
    print('Y Train: ' + str(y_train.shape))

    return x_train, y_train

In [ ]:
'''
Load validation data
'''
def load_validation_data(dir_path):
    df = pd.read_csv('../data/train.csv')
    
    ignore_list = read_ignore_list()
    n_valid_images = 179  #948
    
    image_list = []
    y_list = []
    
    for idx, i in enumerate(os.listdir(validation_dir)):
    #for i in range(0, n_train_images):
        if i not in ignore_list and i.endswith('.png'):
            img_path = os.path.join(dir_path, str(i)) # + '.png'
            img = cv2.imread(img_path)
            print('Image shape: ' + str(img.shape))
            
            image_list.append(img)
            
            row = df.ix[idx]
            
            y_row = np.zeros((5))
            y_row[0] = row['adult_males']
            y_row[1] = row['subadult_males']
            y_row[2] = row['adult_females']
            y_row[3] = row['juveniles']
            y_row[4] = row['pups']
            y_list.append(y_row)
    x_train = np.asarray(image_list)
    y_train = np.asanyarray(y_list)
    
    print('X Train: ' + str(x_train.shape))
    print('Y Train: ' + str(y_train.shape))

    return x_train, y_train

In [ ]:
'''
batch loading data
'''
train_datagen = ImageDataGenerator(rescale=1./255)  # imgs rescaled by 1./255
test_datagen = ImageDataGenerator(rescale=1./255) #rescale=1./255 

In [ ]:
'''
Ensure data is nested in correct structure
data
  |--test/
    |--test/
      |--img1.png
'''
train_generator = train_datagen.flow_from_directory('../data_512/train/', batch_size=16) # train_dir

validation_generator = test_datagen.flow_from_directory('../data_512/validation', batch_size=16)
    
# view shapes of batch
for data_batch, labels_batch in train_generator:
    print('Data batch: ', data_batch.shape)
    print('Labels batch: ', labels_batch.shape)
    break

## Model

In [6]:
'''
Original model w/ dropout layers
'''
def get_dropout_model():
    input_shape = (image_size, image_size, 3)
    
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size = (3, 3),
                    padding = 'same', input_shape = input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    
    model.add(Conv2D(64, kernel_size = (3, 3),
                    padding = 'same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(128, kernel_size = (3, 3),
                    padding = 'same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(n_classes, kernel_size=(3, 3), 
                     padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    
    model.add(GlobalAveragePooling2D())
    
    print(model.summary())
    
    model.compile(loss = keras.losses.mean_squared_error,
                  metrics = ['accuracy'],
                  optimizer = keras.optimizers.Adadelta())  #keras.optimizers.Adam()
    return model

## Training

In [20]:
def train():
    model = get_dropout_model() #get_vgg16_model()
    
    x_train, y_train = load_data('../data_512/')  #'../data_512/' train_dir
    #validation_tuple = load_validation_data(validation_dir)
        
    datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, 
                                 rotation_range=40, width_shift_range=0.2,
                                 height_shift_range=0.2,shear_range=0.2,
                                 zoom_range=0.2, fill_mode='nearest' )
    
    history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), 
                        steps_per_epoch = len(x_train) / batch_size, 
                                  epochs = epochs)  #, validation_data=validation_tuple
    
    model.save('./models/' + model_name + str(datetime.datetime.now())[11:-10] + 'all_100e.h5')
    
    return history

In [21]:
history = train()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 512, 512, 32)      896       
_________________________________________________________________
activation_17 (Activation)   (None, 512, 512, 32)      0         
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 256, 256, 32)      0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 256, 256, 64)      18496     
_________________________________________________________________
activation_18 (Activation)   (None, 256, 256, 64)      0         
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 128, 128, 64)      0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 128, 128, 64)      0         
__________

/home/bss/opt/anaconda3/envs/noaa/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
I

Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
I

Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
I

48/47 [==============================] - 88s - loss: 1839.7397 - acc: 0.4333    
Epoch 2/100
48/47 [==============================] - 26s - loss: 1764.3096 - acc: 0.4797    
Epoch 3/100
48/47 [==============================] - 26s - loss: 1769.2717 - acc: 0.4812    
Epoch 4/100
48/47 [==============================] - 26s - loss: 1781.2330 - acc: 0.4771    
Epoch 5/100
48/47 [==============================] - 26s - loss: 1774.9090 - acc: 0.4827    
Epoch 6/100
48/47 [==============================] - 26s - loss: 1753.8646 - acc: 0.4796    
Epoch 7/100
48/47 [==============================] - 26s - loss: 1775.1827 - acc: 0.4796    
Epoch 8/100
48/47 [==============================] - 26s - loss: 1741.9398 - acc: 0.4781    
Epoch 9/100
48/47 [==============================] - 26s - loss: 1775.0259 - acc: 0.4792    
Epoch 10/100
48/47 [==============================] - 26s - loss: 1794.7173 - acc: 0.4766    
Epoch 11/100
48/47 [==============================] - 26s - loss: 1768.3600 - acc

48/47 [==============================] - 26s - loss: 1789.2407 - acc: 0.4766    
Epoch 89/100
48/47 [==============================] - 26s - loss: 1743.3740 - acc: 0.4781    
Epoch 90/100
48/47 [==============================] - 26s - loss: 1727.9458 - acc: 0.4745    
Epoch 91/100
48/47 [==============================] - 26s - loss: 1729.2828 - acc: 0.4755    
Epoch 92/100
48/47 [==============================] - 26s - loss: 1725.0799 - acc: 0.4781    
Epoch 93/100
48/47 [==============================] - 26s - loss: 1754.7275 - acc: 0.4760    
Epoch 94/100
48/47 [==============================] - 26s - loss: 1740.5728 - acc: 0.4755    
Epoch 95/100
48/47 [==============================] - 26s - loss: 1747.1997 - acc: 0.4766    
Epoch 96/100
48/47 [==============================] - 26s - loss: 1753.7274 - acc: 0.4771    
Epoch 97/100
48/47 [==============================] - 26s - loss: 1733.4139 - acc: 0.4842    
Epoch 98/100
48/47 [==============================] - 26s - loss: 1848.35

## Evaluation

In [ ]:
print(history.history.keys())

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo')
plt.plot(epochs, val_acc, 'b')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo')
plt.plot(epochs, val_loss, 'b')
plt.title('Training and validation loss')

plt.show()
plt.savefig('150e_wDropout_mon.png')

In [ ]:
# summarize history for accuracy
plt.plot(history.history['loss'])
plt.plot(history.history['acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Submission

## Full Data

In [22]:
'''
Removing mismatched images, reading in images w/ corresponding attr
'''
def load_all_data(dir_path):
    df = pd.read_csv('../data/train.csv')
    
    ignore_list = read_ignore_list()
    n_train_images = 948
    
    image_list = []
    y_list = []
    
    #for idx, i in enumerate(os.listdir('../data_512/all_data')):
    for i in range(0, n_train_images):
        #if i not in ignore_list and i.endswith('.png'):
        img_path = os.path.join(dir_path, str(i) + '.png')
        img = cv2.imread(img_path)
        print('Image shape: ' + str(img.shape))

        image_list.append(img)

        row = df.ix[i]

        y_row = np.zeros((5))
        y_row[0] = row['adult_males']
        y_row[1] = row['subadult_males']
        y_row[2] = row['adult_females']
        y_row[3] = row['juveniles']
        y_row[4] = row['pups']
        y_list.append(y_row)
    x_train = np.asarray(image_list)
    y_train = np.asanyarray(y_list)
    
    print('X Train: ' + str(x_train.shape))
    print('Y Train: ' + str(y_train.shape))

    return x_train, y_train

In [ ]:
'''
train model on full data
'''
def train_full():
    model = get_vgg16_model()#get_dropout_model()
    
    x_train, y_train = load_all_data('../data_512/all_data/')  #'../data_512/' train_dir
        
    datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, 
                                 rotation_range=80)
    
    history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=8),steps_per_epoch = len(x_train) / batch_size, epochs = epochs)
    
    model.save('./models/' + model_name + str(datetime.datetime.now())[:-7] + 'transvgg16.h5')
    
    return history

In [ ]:
history = train_full()

In [28]:
def create_submission():
    model = load_model('./models/' + model_name + '21:52all_100e.h5')
    
    n_test_images = 18636
    
    pred_arr = np.zeros((n_test_images, n_classes), np.int32)
    
    for k in range(0, n_test_images):
        image_path = '../data_512/' + str(k) + '.png'
        
        img = cv2.imread(image_path)
        img = img[None, ...]
        pred = model.predict(img).astype(int)
        
        pred_arr[k,:] = pred
    
    print('Pred arr: ' + str(pred_arr.shape))
    
    pred_arr = pred_arr.clip(min=0)
    
    df_submission = pd.DataFrame()
    df_submission['test_id'] = range(0, n_test_images)
    df_submission['adult_males'] = pred_arr[:,0]
    df_submission['subadult_males'] = pred_arr[:,1]
    df_submission['adult_females'] = pred_arr[:,2]
    df_submission['juveniles'] = pred_arr[:,3]
    df_submission['pups'] = pred_arr[:,4]
    
    df_submission.to_csv('./submissions/' + model_name + '_submission.csv', index = False)

In [29]:
create_submission()

Pred arr: (18636, 5)


## Transfer Learning

In [ ]:
from keras.applications import VGG16
import numpy as np

conv_base = VGG16(weights='imagenet',
                 include_top=False,
                 input_shape=(512, 512, 3))

tl_datagen =ImageDataGenerator(rescale=1./255)

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = tl_datagen.flow_from_directory(directory,
                                           target_size = (512, 512),
                                           batch_size=batch_size)
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels

train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)

In [ ]:
# reshape
train_features = np.reshape(train_features, (2000, 4 * 4 *512))
validation_features = np.reshape(validation_features, (1000, 4* 4* 512))
test_features = np.reshape(test_features, (1000, 4*4*512))

In [ ]:
# sample 
from keras.applications import VGG16

def get_vgg16_model():
    batch_size= 8
    vgg16 = keras.applications.vgg16.VGG16(include_top=False, 
                                           weights='imagenet', 
                                           input_shape=(image_size,image_size,3))

    x= Conv2D(n_classes, (1, 1), activation='relu')(vgg16.output)
    x= GlobalAveragePooling2D()(x)

    model = Model(vgg16.input, x)

    print(model.summary())

    history = model.compile(loss=keras.losses.mean_squared_error,
            optimizer= keras.optimizers.Adadelta())

    return model

## Resources

https://www.kaggle.com/c/noaa-fisheries-steller-sea-lion-population-count/

https://www.kaggle.com/radustoicescu/use-keras-to-classify-sea-lions-0-91-accuracy

https://www.kaggle.com/radustoicescu/count-the-sea-lions-in-the-first-image

https://github.com/mrgloom/Kaggle-Sea-Lions-Solution/blob/master/01-Image%20level%20regression/run_me.py

http://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/